In [ ]:
#%% TRY TO SAVE DERIVATIVE FROM BIDS DATASET HERE:
from main_meg_qc import make_derivative_meg_qc
config_file_name = 'settings.ini'
raw = make_derivative_meg_qc(config_file_name)
raw

In [ ]:
l=[]
len(l)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import mne
from mne.time_frequency import tfr_morlet, psd_multitaper, psd_welch
import configparser
import ancpbids
import plotly
import mpld3

from main_meg_qc import make_derivative_meg_qc
from initial_meg_qc import get_all_config_params, sanity_check, initial_processing
from RMSE_meq_qc import RMSE_meg_qc
from PSD_meg_qc import PSD_meg_qc
from Peaks_manual_meg_qc import PP_manual_meg_qc
from Peaks_auto_meg_qc import PP_auto_meg_qc
from ECG_meg_qc import ECG_meg_qc
from EOG_meg_qc import EOG_meg_qc
from universal_html_report import keep_fig_derivs, make_joined_report
import time


In [ ]:
from main_meg_qc import get_all_config_params, initial_processing
all_qc_params=get_all_config_params('settings.ini')
#print(all_qc_params)

data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003483/sub-009/ses-1/meg/sub-009_ses-1_task-deduction_run-1_meg.fif' #GOOD ECG CHANNEL
#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from lab/mikado/sub_HT05ND16/210811/mikado-1.fif' #NO ECG CHANNEL, GOOD RECONSTRUCT
#data_file='/Volumes/M2_DATA/MEG_QC_stuff/data/from lab/forrest_gump_meg/en04ns31_vp15/190524/vp15_block1-1.fif'  #BAD ECG CHANNEL

#dict_of_dfs_epoch, dict_epochs_mg, channels, raw_bandpass, raw_filtered_resampled, raw_cropped, raw, active_shielding_used = initial_processing(default_settings=all_qc_params['default'], filtering_settings=all_qc_params['Filtering'], epoching_params=all_qc_params['Epoching'], data_file=data_file)

raw = mne.io.read_raw_fif(data_file)
raw_cropped = raw.copy()
tmin_my_plot=300
tmax_my_plot=360
duration_my_plot=tmax_my_plot-tmin_my_plot
raw_cropped.crop(tmin=tmin_my_plot, tmax=tmax_my_plot)

#raw_cropped.drop_channels(ECG_channel_name)

raw_cropped

In [ ]:
from ECG_meg_qc import find_ecg_affected_channels

mag_ch_names = raw.copy().pick_types(meg='mag').ch_names if 'mag' in raw else None
grad_ch_names = raw.copy().pick_types(meg='grad').ch_names if 'grad' in raw else None
channels = {'mag': mag_ch_names, 'grad': grad_ch_names}
sfreq=raw.info['sfreq']
m_or_g='mag'
tmin=-0.1
tmax=0.1
ecg_epochs = mne.preprocessing.create_ecg_epochs(raw, picks=channels[m_or_g], tmin=tmin, tmax=tmax)


ecg_affected_channels, fig_affected, fig_not_affected, fig_avg=find_ecg_affected_channels(ecg_epochs, channels, m_or_g, norm_lvl=1, thresh_lvl_peakfinder=5, sfreq=sfreq, tmin=tmin, tmax=tmax, plotflag=True,  use_abs_of_all_data='True')


In [ ]:
ecg_affected_channels, fig_affected, fig_not_affected, fig_avg=find_ecg_affected_channels(ecg_epochs, channels, m_or_g, norm_lvl=1, thresh_lvl_peakfinder=5, sfreq=sfreq, tmin=tmin, tmax=tmax, plotflag=True, use_abs_of_all_data='False')


In [ ]:
ecg_affected_channels, fig_affected, fig_not_affected, fig_avg=find_ecg_affected_channels(ecg_epochs, channels, m_or_g, norm_lvl=1, thresh_lvl_peakfinder=5, sfreq=sfreq, tmin=tmin, tmax=tmax, plotflag=True, use_abs_of_all_data='flip')


In [ ]:
print(len(ecg_affected_channels))


In [ ]:
data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003682/sub-001/ses-01/meg/sub-001_ses-01_task-AversiveLearningReplay_run-01_meg.fif'
raw = mne.io.read_raw_fif(data_file)
raw.info['chs']

In [ ]:


import numpy as np
import mne
y=np.array([1,4,5,0,0,0])
peaks_l, peaks_m = mne.preprocessing.peak_finder(y)

print(peaks_l)
print(type(peaks_l[0]))
print(len(peaks_l))

print(np.array([87]))
print(type(np.array([87])[0]))
#print(len(np.array(87)))


print(np.ndarray(87))
print(len(np.ndarray(87)))

In [ ]:
# Other useful ancp stuff:

config = configparser.ConfigParser()
config.read('settings.ini')

default_direct = config['DEFAULT']['data_directory']
dataset_path = ancpbids.utils.fetch_dataset(default_direct)

from ancpbids import BIDSLayout
layout = BIDSLayout(dataset_path)

list_of_fifs = layout.get(suffix='meg', extension='.fif', return_type='filename')

list_of_subs = layout.get_subjects()


list_of_entities = layout.get_entities()
print(list_of_entities)

In [ ]:
#TRY SEPARATE FUNCS HERE

config = configparser.ConfigParser()
config.read('settings.ini')
data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003483/sub-009/ses-1/meg/sub-009_ses-1_task-deduction_run-1_meg.fif'
all_qc_params = get_all_config_params('settings.ini')
dict_of_dfs_epoch, dict_epochs_mg, channels, raw_filtered, raw_filtered_resampled, raw_cropped, raw, active_shielding_used = initial_processing(default_settings=all_qc_params['default'], filtering_settings=all_qc_params['Filtering'], epoching_params=all_qc_params['Epoching'], data_file=data_file)

m_or_g_chosen = ['mag']
sid='009'


In [ ]:
start_time = time.time()


#dfs_ptp_amlitude_annot, bad_channels, amplit_annot_with_ch_names = PP_auto_meg_qc(sid, config, channels, raw, m_or_g_chosen)

#derivs, big_rmse_with_value_all_data, small_rmse_with_value_all_data = RMSE_meg_qc(all_qc_params['RMSE'], channels, dict_epochs_mg, dict_dict_of_dfs_epoch, raw, m_or_g_chosen)

derivs = PP_manual_meg_qc(all_qc_params['PTP_manual'], channels, dict_epochs_mg, dict_of_dfs_epoch, raw, m_or_g_chosen)

#out_with_name_and_format = PSD_meg_qc(sid, config, channels, raw, m_or_g_chosen)

derivs = ECG_meg_qc(config, raw, m_or_g_chosen)

#out_with_name_and_format = EOG_meg_qc(config, raw, m_or_g_chosen)

#out_with_name_and_format, bad_channels = PP_auto_meg_qc(sid, config, channels, raw, m_or_g_chosen)

print("--- Execution %s seconds ---" % (time.time() - start_time))


for der in derivs:
    if der.content_type == 'plotly':
        der.content.show()


In [ ]:
# out_with_name_and_format[0].convert_fig_to_html()
from universal_plots import QC_derivative
dr = QC_derivative('001', 'mean_EC_epoch', None, '')
# dr_html = dr.convert_fig_to_html()
# l=["a"]

# if dr_html is not None:

#     l += dr_html
    
# print(l)

sec = dr.get_section()
print(sec)


In [ ]:
print(figs)
all_fig_derivs = figs
figures_report = {}
for x in range(0, len(all_fig_derivs)):
    if all_fig_derivs[x][3]=='plotly':
        figures_report["f{0}".format(x)] = plotly.io.to_html(all_fig_derivs[x][0])
    elif all_fig_derivs[x][3]=='matplotlib':
        figures_report["f{0}".format(x)] = mpld3.fig_to_html(all_fig_derivs[x][0]);

print(figures_report)

In [ ]:

import mpld3

html_fig=mpld3.fig_to_html(figs[0][0])
print(html_fig)



# file = open('matpl_fig.html', "w")
# file.write(html_fig)
# file.close()

In [ ]:
html_string = '''
<!doctype html>
<html>
    <head>
        <meta charset="UTF-8">
        <title>MEG QC: Frequency spectrum Report</title>
        <style>body{ margin:0 100;}</style>
    </head>
    
    <body style="font-family: Arial">
        <center>
        <h1>MEG data quality analysis report</h1>
        <br></br>
        <!-- *** Section 1 *** --->
        <h2>Frequency spectrum per channel</h2>
        ''' + html_fig + '''
        <p>graph description...</p>
        </center>
    
    </body>
</html>'''

with open('report_trial.html', 'w', encoding = 'utf8') as f:
    f.write(html_string)

In [ ]:
#config = configparser.ConfigParser()
#config.read('settings.ini')

#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds000117/sub-01/ses-meg/meg/sub-01_ses-meg_task-facerecognition_run-01_meg.fif'
# file does not start with a file id tag

#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003392/sub-01/meg/sub-01_task-localizer_meg.fif'
# SSS frilter. need to allow maxshiled.

#data_file ='/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003694/sub-01/meg/sub-01_task-MEM_run-01_meg.fif'
#raw = mne.io.read_raw_fif(data_file, on_split_missing='ignore')


#dict_of_dfs_epoch, dict_epochs_mg, channels, raw_bandpass, raw_filtered_resampled, raw_cropped, raw = initial_processing(config, data_file)

#data_file ='/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003922/sub-Mp150285/ses-01/meg/sub-Mp150285_ses-01_acq-crosstalk_meg.fif'
#Could not find measurement data: how many in this set, which subjects?


data_file ='/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds004229/sub-102/meg/sub-102_task-amnoise_meg.fif'
#SSS filter
raw = mne.io.read_raw_fif(data_file, allow_maxshield=True)


#data_dir ='/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/not fitting no fif/ds000246/sub-0001/meg/sub-0001_task-AEF_run-01_meg.ds'
#raw = mne.io.read_raw_ctf(data_dir)


In [ ]:
import configparser
from ancpbids import BIDSLayout

config = configparser.ConfigParser()
config.read('settings.ini')

default_section = config['DEFAULT']

m_or_g_chosen = default_section['do_for'] 
m_or_g_chosen = m_or_g_chosen.replace(" ", "")
m_or_g_chosen = m_or_g_chosen.split(",")
#m_or_g_chosen = select_m_or_g(default_section)

dataset_path = default_section['data_directory']

layout = BIDSLayout(dataset_path)
schema = layout.schema

#create derivative folder first!
derivative = layout.dataset.create_derivative(name="Meg_QC")
derivative.dataset_description.GeneratedBy.Name = "MEG QC Pipeline"

list_of_subs = layout.get_subjects()
#print(list_of_subs)
for sub in list_of_subs:
    list_of_fifs_per_sub = layout.get(suffix='meg', extension='.fif', sub=sub)
    for_entities={}
    for file in list_of_fifs_per_sub:
        k=file['entities'][1]['key']
        v=file['entities'][1]['value']
        for_entities[k]=v
    print(for_entities)

#print(list_of_fifs_per_sub)
# for sid in [list_of_subs[0]]: #RUN OVER JUST 1 SUBJ
# #for sid in list_of_subs: 

#     subject_folder = derivative.create_folder(type_=schema.Subject, name='sub-'+sid)

#     list_of_fifs = layout.get(suffix='meg', extension='.fif', return_type='filename', subj=sid)
#     #Devide here fifs by task, ses , run

#     for data_file in [list_of_fifs[0]]: #RUN OVER JUST 1 FIF because is not divided by tasks yet..
#         print(type(data_file))